<h3>Produced By: Garrett Zimmerman</h3>
<h3>Version 1: January 4th, 2024</h3>
<h2>Goal:</h2> 
<p>Automate Youtube Upload process. I would like a python script that will grab a video file and upload it to my account. Make sure your code is optimized for performance. I would like to see a basic demo of the script upload youtube videos to your personal account. This is going to be very important for one of our products and our business operations. We will extend the framework to any social media platform</p>

<h2>Action Plan:</h2>
<ol>
    <li>Grab a video/videos file from a folder</li>
    <li>Upload the video file to youtube account</li>
    <li>Optimize the process</li>
</ol>

<h1>Actions Items Needed to Complete before continuing:</h1>
<dl>
    <dt>1. Google Developer Console Account:</dt> 
        <dd>A:  You need a Google account to access the Google Developer Console. (COST $25 for life)</dd>
    <dt>2. YouTube Data API Key: </dt>
        <dd>You'll need to create a project in the Google Developer Console and enable the YouTube Data API for that project.</dd>
        <dd>A:  Sign in to the Google Developer Console</dd>
        <dd>B:  Go to the Google Developer Console and sign in with your Google account.</dd>
        <dd>C:  Once you're logged in, create a new project by clicking on "Select a project" or "New Project" at the top of the page.</dd>
        <dd>D:  Give your project a name and create it.</dd>
        <dd>E:  Enable the YouTube Data API for Your Project</dd>
        <dd>F:  With the project selected, navigate to the "Library" in the Google Developer Console.</dd>
        <dd>G:  Search for "YouTube Data API v3" and select it.</dd>
        <dd>H:  Click on "Enable" to add the YouTube Data API to your project. This step is crucial as it allows your application to interact with YouTube.</dd>
    <dt>3. OAuth 2.0 Client ID: For uploading videos, you need to authenticate via OAuth 2.0. In the Google Developer Console, create OAuth 2.0 credentials.</dt>
        <dd>A:  Configure the OAuth Consent Screen:</dd>
        <dd>B:  In the Developer Console, go to the "OAuth consent screen" section.</dd>
        <dd>C:  Select the user type (usually "External" for apps that are available to any user with a Google account).</dd>
        <dd>D:  Fill in the required fields like the app name, user support email, and developer contact information. This information will be shown to users when they are asked to grant permissions to your app.</dd>
        <dd>E:  Create OAuth 2.0 Credentials:</dd>
        <dd>F:  Go to the "Credentials" tab in the Developer Console.</dd>
        <dd>G:  Click on "Create Credentials" and choose "OAuth client ID".</dd>
        <dd>H:  You might need to configure the consent screen before you can create credentials.</dd>
        <dd>I:  Add authorized redirect URIs if needed (this is essential for web applications).</d>
        <dd>J:  Obtain the Client ID and Client Secret:</dd>
        <dd>K:  Once you create the OAuth client ID, you'll receive a client ID and client secret.</dd> 
        <dd>L:  These are important for your Python script to authenticate with Google's servers.</dd>
        <dd>M:  Download the JSON Configuration File:</dd>
        <dd>N:  You can download the configuration file that contains your client ID and client secret. This file is often named client_secrets.json.</dd>
        <dd>O:  This file will be used in your Python script to facilitate OAuth 2.0 authentication</dd>
</dl>

In [ ]:
#All modules needed
import os
from typing import List
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import googleapiclient.http
import pandas as pd
import pkg_resources



<h2>The function find_video_files will take in a folder path and return a the list of videos in that file by looking for .mp4</h2>
<p>Make sure and change the file location if need be</p>

In [ ]:
def find_video_files(folder_path: str, file_extension: str = ".mp4") -> List[str]:
    """
    Finds video files in the specified folder with the given file extension.

    :param folder_path: Path to the folder where video files are located.
    :param file_extension: The extension of the video files to find.
    :return: List of video file paths.
    """
    try:
        video_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(file_extension)]
        assert len(video_files) > 0, "No video files found in the folder."
        return video_files
    except Exception as e:
        print(f"Error finding video files: {e}")
        return []


<h1>Every Video Updated to Youtube will need to have this information:</h1>
<ul>
    <li>title </li> 
    <li>description</li> 
    <li>category </li> 
    <li>tags </li> 
    <li>privacy status </li> 
</ul>

<h1>Two Categories require specified responses</h1>
<h2>Privacy status:</h2>
<dl>
    <dt>'public'</dt> 
        <dd>The video can be viewed by anyone </dd>
    <dt>'private'</dt> 
        <dd>The video can only be viewed by the uploader and the users specifically granted permission</dd>
    <dt>'unlisted'</dt> 
        <dd>The video will not appear in any of YouTube's public spaces, but anyone with the link can view it</dd>
</dl>
<h2>Category</h2>
<p>Category Must be choosen from a list of numbers representing the catergory. Run the cell below to see a table of avaliable categories.</p>  

In [ ]:
# Data for YouTube category IDs
category_data = {
    "Category ID": [
        "1", "2", "10", "15", "17", "18", "19", "20", 
        "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"
    ],
    "Category Name": [
        "Film & Animation", "Autos & Vehicles", "Music", "Pets & Animals",
        "Sports", "Short Movies", "Travel & Events", "Gaming",
        "Videoblogging", "People & Blogs", "Comedy", "Entertainment",
        "News & Politics", "Howto & Style", "Education", "Science & Technology",
        "Nonprofits & Activism", "Movies"
    ]
}

# Create a DataFrame
df = pd.DataFrame(category_data)

# Display the DataFrame
df

In [ ]:
#This was the first function created. There are some speed improvements below:

def upload_video(file_path):
    # User inputs for video details
    title = input(f"Enter the title for the video '{os.path.basename(file_path)}': ")
    description = input("Enter the description for the video: ")
    category = input("Enter the category ID for the video (e.g., '22' for People & Blogs): ")
    tags = input("Enter the tags for the video (comma-separated): ").split(',')
    privacy_status = input("Enter the privacy status (public, private, or unlisted): ")

    assert privacy_status in ['public', 'private', 'unlisted'], "Invalid privacy status."

    # OAuth 2.0 setup
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for local testing!
    client_secrets_file = "client_secrets.json"

    scopes = ["https://www.googleapis.com/auth/youtube.upload"]
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes=scopes)

    # Using run_local_server
    credentials = flow.run_local_server()

    youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)


    request_body = {
        "snippet": {
            "categoryId": category,
            "title": title,
            "description": description,
            "tags": tags
        },
        "status": {
            "privacyStatus": privacy_status
        }
    }

    media_file = googleapiclient.http.MediaFileUpload(file_path)

    # Call the API to upload the video
    request = youtube.videos().insert(
        part="snippet,status",
        body=request_body,
        media_body=media_file
    )

    response = request.execute()
    print(f"Upload successful for {file_path}.")

<h2>Uploading Videos From file:</h2>

In [ ]:

#Enter the folder where the videos are
folder_path = "D:\Test_videos"  # Replace with your actual folder path
video_files = find_video_files(folder_path)

print(video_files)  # This will print the list of found video files
for video_file in video_files:
    upload_video(video_file)
    print("Video Uploaded:",video_file)

<h2>Creating a requirements txt</h2>

In [ ]:
packages = ["google-auth-oauthlib", "google-api-python-client", "pandas"]
with open("requirements.txt", "w") as f:
    for package in packages:
        version = pkg_resources.get_distribution(package).version
        f.write(f"{package}=={version}\n")

<h2>Speed Improvements</h2>
<h3>Separate OAuth Initialization from Upload Function</h3>
<p> Initialize the OAuth flow and YouTube client once, outside the upload_video function. This prevents re-authenticating and rebuilding the client for each video upload, which is time-consuming and inefficient.</p>

In [ ]:
def initialize_youtube_client():
    try:
        os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # For local testing only!
        client_secrets_file = "client_secrets.json"
        scopes = ["https://www.googleapis.com/auth/youtube.upload"]
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
            client_secrets_file, scopes=scopes)
        credentials = flow.run_local_server()
        return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)
    except Exception as e:
        print(f"Error initializing YouTube client: {e}")
        return None

In [ ]:
def upload_video(youtube, file_path, title, description, category, tags, privacy_status):
    try:
        request_body = {
            "snippet": {
                "categoryId": category,
                "title": title,
                "description": description,
                "tags": tags
            },
            "status": {
                "privacyStatus": privacy_status
            }
        }

        media_file = googleapiclient.http.MediaFileUpload(file_path)

        request = youtube.videos().insert(
            part="snippet,status",
            body=request_body,
            media_body=media_file
        )

        response = request.execute()
        print(f"Upload successful for {file_path}. Response:", response)
    except googleapiclient.errors.HttpError as e:
        print(f"An HTTP error occurred: {e.resp.status} {e.content}")
    except Exception as e:
        print(f"An error occurred during video upload: {e}")

In [ ]:
def main():
    folder_path = "D:\\Test_videos"  # Use double backslashes for Windows paths
    video_files = find_video_files(folder_path)

    youtube = initialize_youtube_client()
    if youtube is None:
        return

    for video_file in video_files:
        title = input(f"Enter the title for the video '{os.path.basename(video_file)}': ")
        description = input("Enter the description for the video: ")
        category = input("Enter the category ID for the video: ")
        tags = input("Enter the tags for the video (comma-separated): ").split(',')
        privacy_status = input("Enter the privacy status (public, private, or unlisted): ")

        upload_video(youtube, video_file, title, description, category, tags, privacy_status)
        print("Video Uploaded:", video_file)

if __name__ == "__main__":
    main()

<h2>Further Improvemnts and Considerations</h2>
<dl>
    <dt>User Input</dt> 
    <dd>Currently, the script still asks for input for each video. Depending on our use case, we might automate this or use a different approach to handle metadata.<dd>
</dl>